<a href="https://colab.research.google.com/github/bibersay/Deep-Project/blob/main/Scream_Distinction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd '/gdrive/MyDrive/Deep-Project'
!mkdir -p /content/dataset
!cp scream.zip /content/dataset/
%cd '/content/dataset/'
!unzip -q scream.zip

Mounted at /gdrive
/gdrive/MyDrive/Deep-Project
/content/dataset


In [ ]:
!sudo apt-get install libportaudio2
!pip install sounddevice
import librosa
from matplotlib import pyplot as plt
import sounddevice as sd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (817 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [ ]:
from os import listdir
from os.path import join, splitext
import time
import math
from tqdm import tqdm
from torch.utils.data   import Dataset, DataLoader
import pandas as pd
import numpy as np

In [ ]:
list_file_fn = 'file_list.xlsx'

with pd.ExcelWriter(list_file_fn) as writer:
    audio_names = []
    for i , file_name in enumerate(label_files):
        temp = pd.read_csv(label_path + file_name, sep = '\t', header = None)
        temp.columns = ['start', 'end', 'label']
        temp.to_excel(writer , index=False, sheet_name = str(i))
        audio_names.append(file_name[:-3] + 'wav')
        file_name
        if i == len(label_files) -1 :
            pd.DataFrame(audio_names, columns = ['audio_name']).to_excel(writer, sheet_name = 'list')

In [ ]:
def load_label_data(file_path, i, frame_size):
    audio_label = pd.read_excel(file_path, sheet_name = str(i))
    audio_label['label'] = audio_label['label'].apply(lambda x : int(str(x)[0]))
    audio_label = np.array(audio_label)
    audio_label = audio_label[(audio_label[:,1] - audio_label[:,0]) >= frame_size]
    return audio_label

def load_sound_data(file_path, sampling_rate):
    audio, sr = librosa.load(file_path, sr = sampling_rate)
    print(audio, sr)
    return audio, sr

In [ ]:
def process_frame(audio, sr, label, frame_size =1, stride = .2):
    n = np.sum(np.floor(((label[:,1] - label[:,0]) - frame_size) / stride + frame_size))
  
    frame_vector = np.zeros((int(n), frame_size * sr))  
    target_vector = np.zeros((int(n), 1)) 
 
    i = 0
    for temp_label in label:
        start = temp_label[0] 
        end = temp_label[1] 
        y = temp_label[2]  
 
        eter = int(
            np.floor((((end - start) - frame_size) / stride) + 1)
        )  
        for j in range(eter):  
            start_idx = int((start + j * stride) * sr)
            end_idx = start_idx + frame_size * sr
            frame_vector[i] = audio[start_idx:end_idx]  
            target_vector[i] = y  
            i += 1
    return frame_vector, target_vector

In [ ]:
def extract_mel_feature(frame_vector, sr, n_mels, st = 512):
    mel_feature = librosa.feature.melspectrogram(
        y = frame_vector, sr= sr, n_mels= n_mels , hop_length = st
    )
    mel_feature = librosa.core.power_to_db(mel_feature, ref = np.max)
    return mel_feature

In [ ]:
sr = 22050
frame_size = 1
stride = .2
n_mels = 64
n_feature = 34
short_time = 512

file_list = pd.read_excel('/content/dataset/file_list.xlsx', sheet_name='list')

frame_vectors = []
label_vectors = []
idx_count = []

for file_i, file_name in enumerate(file_list['audio_name']):
    if file_i % 10 ==0:
        print(f'in processing {file_i}th file ')
    
    label_data_temp = load_label_data('/content/dataset/file_list.xlsx', file_i, frame_size)

    if len(label_data_temp) == 0 :
           idx_count.append(0)
           continue

    audio_path = '/content/dataset/data/raw_data/' + file_name
    audio_data_temp, sr  = load_sound_data(audio_path, sr)
    frame_vector, label_vector = process_frame(
        audio = audio_data_temp, sr=sr, label = label_data_temp, frame_size = frame_size, stride = stride
    )
    frame_vectors.append(frame_vector)
    label_vectors.append(label_vector)

    idx_count.append(frame_vector.shape[0])

frame_vectors = np.concatenate(frame_vectors)
label_vectors = np.concatenate(label_vectors)


in processing 0th file 
[0. 0. 0. ... 0. 0. 0.] 22050
[-0.27773625 -0.32427642 -0.14350182 ... -0.18024364 -0.27594572
 -0.3596239 ] 22050
[0. 0. 0. ... 0. 0. 0.] 22050
[-0.0138855  -0.01092529 -0.00866699 ...  0.00097656  0.0012207
  0.00128174] 22050
[ 0.          0.          0.         ... -0.0079037  -0.05037875
  0.        ] 22050
[0.         0.         0.         ... 0.00013539 0.00012781 0.00012629] 22050
[ 0.          0.          0.         ...  0.00510935 -0.00171055
 -0.00892943] 22050
[0. 0. 0. ... 0. 0. 0.] 22050
in processing 10th file 
[-6.5481025e-01 -4.3213984e-01 -8.0406778e-03 ... -4.0445913e-05
  4.3613333e-04  0.0000000e+00] 22050
[0. 0. 0. ... 0. 0. 0.] 22050
[0. 0. 0. ... 0. 0. 0.] 22050
[0. 0. 0. ... 0. 0. 0.] 22050
[0.         0.         0.         ... 0.00028551 0.00021983 0.        ] 22050
[0.23730469 0.20471191 0.19433594 ... 0.00785828 0.00283813 0.01673889] 22050
[ 2.9607375e-07 -5.1644606e-06  7.7028762e-06 ...  2.5339868e-05
  3.5420213e-05  0.0000000e+00

In [ ]:
mel_features = []
for frame_vector in frame_vectors:
    mel_feature = extract_mel_feature(
        frame_vector = frame_vector, sr = sr, n_mels = n_mels
    )
    mel_features.append(mel_feature)
mel_features = np.stack(mel_features)
print(f'y shpae : {y.shape}\n scream sount : {sum(y==1)}\n no scream : {sum(y==0)}')

y shpae : (22050,)
 scream sount : 0
 no scream : 0


In [ ]:
train_ratio = .7

total_n = frame_vectors.shape[0]
train_n = int(total_n * train_ratio)

train_idxes = np.random.choice(total_n, train_n, replace = False)
train_mask = np.zeros(shape = total_n, dtype = bool)
train_mask[train_idxes] = True

In [ ]:
train_mel_features = mel_features[train_mask]
valid_mel_features = mel_features[~train_mask]

train_label_vectors = label_vectors[train_mask]
valid_label_vectors = label_vectors[~train_mask]

In [ ]:
import torch
from torch import nn, optim
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
class ScreamDataset(Dataset):
    def __init__(self, x, y):
        self.audio_features = x
        self.labels = y

    def __getitem__(self, idx):
        audio_features = torch.FloatTensor(self.audio_features[idx])
        audio_features = audio_features.unsqueeze(0)
        label = self.labels[idx]
        return (audio_features, label)

    def __len__(self):
        return len(self.audio_features)

In [ ]:
batch_size = 128
learning_rate = 0.001
epochs = 10

In [ ]:
train_dataset = ScreamDataset(train_mel_features, train_label_vectors)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

valid_dataset = ScreamDataset(valid_mel_features, valid_label_vectors)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = nn.Sequential(
    # input = (n,1,64,44) (batchsize, channel, height, width)
    nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = (64,1),),
    # x = (n, 32, 1, 44)
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Dropout2d(p=.3),
    nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (1,9), stride =4),
    # x = (n, 64, 1, 9)
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Dropout2d(p=.3),
    nn.Flatten(),
    nn.Linear(64 *1 * 9, 1)
).to(device)
print(device)

cpu


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
n_epochs = 100
train_n = len(train_dataset)
valid_n = len(valid_dataset)

for e in range(1, n_epochs +1):
    train_loss = 0
    train_acc = 0
    train_f1_score = 0
    valid_loss = 0
    valid_acc = 0
    valid_f1_score = 0

    model.train()

    for audio_feature, label in train_loader:
        audio_feature = audio_feature.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        pred = model(audio_feature)
        
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()

        pred = torch.round(torch.sigmoid(pred))
        pred = pred.cpu().detach().numpy()
        label = label.cpu().detach().numpy()

        acc = accuracy_score(label, pred)
        f1 = f1_score(label,pred)

        n = len(label)
        train_loss += loss.item() * n
        train_acc += acc.item()*n
        train_f1_score += f1*n_epochs

    model.eval()
    with torch.no_grad():
        for audio_feature, label in valid_loader :
            audio_feature = audio_feature.to(device)
            label = label.to(device)

            pred = model(audio_feature)
            loss = criterion(pred, label)

            pred = torch.round(torch.sigmoid(pred))
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()

            acc = accuracy_score(label, pred)
            f1 = f1_score(label,pred)

            n = len(label)
            valid_loss += loss.item() * n
            valid_acc += acc.item()*n
            valid_f1_score += f1*n_epochs


    train_loss = f'{train_loss / train_n : .4f}'
    train_acc = f'{train_acc / train_n : .3f}'
    train_f1_score = f'{train_f1_score / train_n : .3f}'    
    valid_loss = f'{valid_loss / valid_n : .4f}'
    valid_acc = f'{valid_acc / valid_n : .3f}'
    valid_f1_score = f'{valid_f1_score / valid_n : .3f}'

    # log metrics
    print(f'Epoch {e+0 : 03}')
    print('      |   loss  |   acc  |   f1   |')
    print(f'TRAIN | {train_loss} | {train_acc} | {train_f1_score} |')
    print(f'VALID | {valid_loss} | {valid_acc} | {valid_f1_score} |')
    print('----------------------------------')
    print()
        


Epoch  01
      |   loss  |   acc  |   f1   |
TRAIN |  0.7359 |  0.470 |  0.280 |
VALID |  0.7250 |  0.421 |  0.245 |
----------------------------------

Epoch  02
      |   loss  |   acc  |   f1   |
TRAIN |  0.7387 |  0.461 |  0.276 |
VALID |  0.7267 |  0.410 |  0.236 |
----------------------------------

Epoch  03
      |   loss  |   acc  |   f1   |
TRAIN |  0.7426 |  0.452 |  0.270 |
VALID |  0.7252 |  0.417 |  0.237 |
----------------------------------

Epoch  04
      |   loss  |   acc  |   f1   |
TRAIN |  0.7365 |  0.455 |  0.276 |
VALID |  0.7273 |  0.411 |  0.233 |
----------------------------------

Epoch  05
      |   loss  |   acc  |   f1   |
TRAIN |  0.7387 |  0.457 |  0.276 |
VALID |  0.7269 |  0.407 |  0.234 |
----------------------------------

Epoch  06
      |   loss  |   acc  |   f1   |
TRAIN |  0.7369 |  0.461 |  0.277 |
VALID |  0.7252 |  0.417 |  0.230 |
----------------------------------

Epoch  07
      |   loss  |   acc  |   f1   |
TRAIN |  0.7393 |  0.456 |  0.

In [ ]:
!pip install kaggle
from google.colab import files, drive
#구글 드라이브 마운트
drive.mount('/gdrive')

model_path = '/content/model/model.pth'
torch.save(model, model_path)
new_model = torch.load(model_path, map_location='cpu')

%cd /content/
!mkdir -p /content/model/

%cd /content/model/
!cp model.pth /gdrive/MyDrive/Deep-Project/

/content/model
